In [2]:
!pip install torch torchvision torchaudio

In [6]:
!pip install xgboost

   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.5/150.0 MB 1.7 MB/s eta 0:01:29
   ---------------------------------------- 1.0/150.0 MB 2.3 MB/s eta 0:01:06
    --------------------------------------- 2.4/150.0 MB 3.9 MB/s eta 0:00:38
   - -------------------------------------- 5.0/150.0 MB 6.2 MB/s eta 0:00:24
   -- ------------------------------------- 8.7/150.0 MB 8.4 MB/s eta 0:00:17
   --- ------------------------------------ 12.8/150.0 MB 11.2 MB/s eta 0:00:13
   --- ------------------------------------ 12.8/150.0 MB 11.2 MB/s eta 0:00:13
   --- ------------------------------------ 12.8/150.0 MB 11.2 MB/s eta 0:00:13
   ------ --------------------------------- 23.9/150.0 MB 14.2 MB/s eta 0:00:09
   -------- ------------------------------- 30.7/150.0 MB 15.1 MB/s eta 0:00:

In [20]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
import xgboost as xgb
import pandas as pd

In [22]:
df = pd.read_csv('C:\\Users\\user\\Downloads\\ParisHousing.csv', encoding='ISO-8859-1', low_memory=False)
Session = sessionmaker(bind=engine)
session = Session()

In [120]:
df.head()

,squareMeters,numberOfRooms,hasYard,hasPool,floors,cityCode,cityPartRange,numPrevOwners,made,isNewBuilt,hasStormProtector,basement,attic,garage,hasStorageRoom,hasGuestRoom,price
0,75523,3,0,1,63,9373,3,8,2005,0,1,4313,9005,956,0,7,7559081.5
1,80771,39,1,1,98,39381,8,6,2015,1,0,3653,2436,128,1,2,8085989.5
2,55712,58,0,1,19,34457,6,8,2021,0,0,2937,8852,135,1,9,5574642.1
3,32316,47,0,0,6,27939,10,4,2012,0,1,659,7141,359,0,3,3232561.2
4,70429,19,1,1,90,38045,3,7,1990,1,0,8435,2429,292,1,4,7055052.0


In [122]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   squareMeters       10000 non-null  int64  
 1   numberOfRooms      10000 non-null  int64  
 2   hasYard            10000 non-null  int64  
 3   hasPool            10000 non-null  int64  
 4   floors             10000 non-null  int64  
 5   cityCode           10000 non-null  int64  
 6   cityPartRange      10000 non-null  int64  
 7   numPrevOwners      10000 non-null  int64  
 8   made               10000 non-null  int64  
 9   isNewBuilt         10000 non-null  int64  
 10  hasStormProtector  10000 non-null  int64  
 11  basement           10000 non-null  int64  
 12  attic              10000 non-null  int64  
 13  garage             10000 non-null  int64  
 14  hasStorageRoom     10000 non-null  int64  
 15  hasGuestRoom       10000 non-null  int64  
 16  price              1000

In [118]:
# Средняя цена на жилье
average_price = y.mean()
average_price

4993447.52575

In [18]:
# Выделение признаков и целевой переменной
X = df.drop(columns='price')
y = df['price']

In [24]:
# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [26]:
# Масштабирование данных
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_scaled[:5], y_train[:5].values

(array([[ 1.1189976 ,  0.22279829, -1.01460667,  1.00652126, -1.66466359,
          0.10022454,  0.51428242, -1.57845239, -1.66971495, -0.99476371,
         -0.99850112,  0.39946723,  1.70579207,  0.70692003,  1.00025003,
         -0.93962897],
        [ 1.02094519,  1.2987203 , -1.01460667, -0.99352099,  0.99630997,
          0.97939221,  0.51428242,  0.16916983, -1.56238092,  1.00526385,
          1.00150113,  0.302882  ,  0.35353226,  0.72979271, -0.99975003,
         -0.62348178],
        [ 0.99548658,  0.70869855,  0.98560362,  1.00652126,  1.58379764,
         -0.36777924,  1.55727534,  1.5672676 , -1.66971495,  1.00526385,
          1.00150113, -0.17900188, -0.9949339 , -0.84460972,  1.00025003,
          0.32495979],
        [-1.18537417, -0.88783089,  0.98560362, -0.99352099,  0.16691561,
         -1.13644986, -1.57170341,  0.51869427,  0.1549635 , -0.99476371,
          1.00150113,  1.1492188 , -1.19910065,  1.17580986,  1.00025003,
          0.95725417],
        [-1.62440385

In [28]:
# Преобразование данных в тензоры
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

In [30]:
# Настройка XGBoost
dtrain = xgb.DMatrix(X_train_scaled, label=y_train)
dtest = xgb.DMatrix(X_test_scaled, label=y_test)

In [140]:
# Определение параметров
params = {
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
    'learning_rate': 0.1,
    'max_depth': 3,
    'min_child_weight': 10,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'seed': 42
}

In [126]:
# Регуляризация
params['reg_alpha'] = 0.1
params['reg_lambda'] = 1.0

In [34]:
# Запуск TensorBoard
writer = SummaryWriter(log_dir='./logs/xgboost')

In [142]:
# Обучение модели и логирование метрик
evals_result = {}
xgb_model = xgb.train(params, dtrain, num_boost_round=1000, evals=evals,
                       early_stopping_rounds=50, verbose_eval=50,
                       evals_result=evals_result)

[0]	train-rmse:2575440.58710	eval-rmse:2672583.09871
[50]	train-rmse:61915.40961	eval-rmse:68162.27948
[100]	train-rmse:27876.74817	eval-rmse:28908.49645
[150]	train-rmse:25059.08286	eval-rmse:25832.81817
[200]	train-rmse:23806.28812	eval-rmse:24740.07939
[250]	train-rmse:22606.69665	eval-rmse:23774.99675
[300]	train-rmse:21667.14766	eval-rmse:23031.44248
[350]	train-rmse:20596.26026	eval-rmse:22248.08679
[400]	train-rmse:19817.06038	eval-rmse:21794.81881
[450]	train-rmse:19082.50246	eval-rmse:21352.06926
[500]	train-rmse:18408.91809	eval-rmse:20973.85924
[550]	train-rmse:17947.02873	eval-rmse:20703.98180
[600]	train-rmse:17290.96618	eval-rmse:20203.02054
[650]	train-rmse:16781.11223	eval-rmse:19910.56975
[700]	train-rmse:16292.83836	eval-rmse:19621.39535
[750]	train-rmse:15834.30266	eval-rmse:19364.63996
[800]	train-rmse:15402.13873	eval-rmse:19123.94979
[850]	train-rmse:15051.78900	eval-rmse:18945.17614
[900]	train-rmse:14705.57467	eval-rmse:18777.16444
[950]	train-rmse:14339.92192	e

In [52]:
# Логирование метрик в TensorBoard
for i in range(len(evals_result['train']['rmse'])):
    writer.add_scalar('Loss/train', evals_result['train']['rmse'][i], i)
    writer.add_scalar('Loss/eval', evals_result['eval']['rmse'][i], i)

In [134]:
xgb_model.best_score

18416.78495681251

In [56]:
y.describe()

count    1.000000e+04
mean     4.993448e+06
std      2.877424e+06
min      1.031350e+04
25%      2.516402e+06
50%      5.016180e+06
75%      7.469092e+06
max      1.000677e+07
Name: price, dtype: float64

In [144]:
writer.close()